## Todo el contenido JSON

In [ ]:
import requests
from bs4 import BeautifulSoup
import json

url = "https://es.wikipedia.org/w/api.php"

# Función para obtener y limpiar el contenido de una sección específica
def obtener_contenido_seccion(seccion_numero, seccion_nombre):
    params = {
        "action": "parse",
        "page": "Cristiano Ronaldo",
        "format": "json",
        "prop": "text",
        "section": seccion_numero
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        content_html = data["parse"]["text"]["*"]
        soup = BeautifulSoup(content_html, "html.parser")
        
        # Elimina referencias y notas adicionales
        for ref in soup.find_all(['sup', 'span', 'a'], {'class': ['reference', 'mw-editsection']}):
            ref.decompose()
        
        # Convierte el contenido a texto limpio y unifica en un solo párrafo
        clean_text = " ".join(soup.get_text(separator="\n").split())
        
        # Retorna el contenido en un formato estructurado
        return {"seccion": seccion_nombre, "contenido": clean_text}
    else:
        print(f"Error en la solicitud: {response.status_code}")
        return None

# Obtener el título de la página
params_title = {
    "action": "query",
    "titles": "Cristiano Ronaldo",
    "format": "json"
}
response_title = requests.get(url, params=params_title)
if response_title.status_code == 200:
    data_title = response_title.json()
    pages = data_title["query"]["pages"]
    page_id = next(iter(pages))
    titulo_pagina = pages[page_id]["title"]
else:
    titulo_pagina = "Cristiano Ronaldo"

# Obtener la lista de secciones
params_sections = {
    "action": "parse",
    "page": "Cristiano Ronaldo",
    "format": "json",
    "prop": "sections"
}
response_sections = requests.get(url, params=params_sections)
if response_sections.status_code == 200:
    data_sections = response_sections.json()
    secciones = data_sections["parse"]["sections"]
else:
    print(f"Error al obtener las secciones: {response_sections.status_code}")
    secciones = []

# Lista de secciones a excluir
secciones_excluir = ["Véase también", "Filmografía", "Notas", "Referencias", "Bibliografía", "Enlaces externos"]

# Iterar sobre todas las secciones y obtener su contenido, excluyendo las especificadas
contenido_secciones = []
for seccion in secciones:
    seccion_numero = seccion["index"]
    seccion_nombre = seccion["line"]
    if seccion_nombre not in secciones_excluir:
        contenido = obtener_contenido_seccion(seccion_numero, seccion_nombre)
        if contenido:
            contenido_secciones.append(contenido)

# Estructura el contenido en un JSON con todas las secciones e incluye el título
json_data2 = {
    "titulo": titulo_pagina,
    "secciones": contenido_secciones
}

# Convierte el diccionario a JSON
json_output2 = json.dumps(json_data, ensure_ascii=False, indent=4)

# Imprime o guarda el JSON estructurado
print(json_output2)


In [ ]:
# Guardar el diccionario en un archivo JSON local
with open('cristiano_ronaldo_test_f.json', 'w', encoding='utf-8') as f:
    json.dump(json_data2, f, ensure_ascii=False, indent=4)

print("El archivo 'cristiano_ronaldo_test_f.json' ha sido guardado exitosamente.")

In [ ]:
import boto3
s3 = boto3.client('s3')

# Especificar el nombre del bucket y el nombre del archivo en S3
bucket_name = 'leonardo-devops-days-staging'  # Reemplaza con el nombre de tu bucket
s3_file_name = 'cristiano_ronaldo_test_f.json'

# Subir el JSON al bucket de S3
try:
    s3.put_object(Bucket=bucket_name, Key=s3_file_name, Body=json_output, ContentType='application/json')
    print(f"El archivo '{s3_file_name}' ha sido subido exitosamente al bucket '{bucket_name}'.")
except NoCredentialsError:
    print("Error: No se encontraron las credenciales de AWS.")
except Exception as e:
    print(f"Ocurrió un error al subir el archivo: {e}")

## secciones particulares

In [ ]:
# Función para obtener y limpiar el contenido de una sección específica
def obtener_contenido_seccion(seccion_numero, seccion_nombre):
    params = {
        "action": "parse",
        "page": "Cristiano Ronaldo",
        "format": "json",
        "prop": "text",
        "section": seccion_numero
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        content_html = data["parse"]["text"]["*"]
        soup = BeautifulSoup(content_html, "html.parser")
        
        # Elimina referencias y notas adicionales
        for ref in soup.find_all(['sup', 'span', 'a'], {'class': ['reference', 'mw-editsection']}):
            ref.decompose()
        
        # Convierte el contenido a texto limpio y unifica en un solo párrafo
        clean_text = " ".join(soup.get_text(separator="\n").split())
        
        # Retorna el contenido en un formato estructurado
        return {"club": seccion_nombre, "historia": clean_text}
    else:
        print(f"Error en la solicitud: {response.status_code}")
        return None

# Obtiene el contenido de la sección "Sporting C. P."
contenido_sporting = obtener_contenido_seccion(3, "Sporting C. P.")
# Obtiene el contenido de la sección "Manchester United F. C."
contenido_manchester = obtener_contenido_seccion(4, "Manchester United F. C.")

# Estructura el contenido en un JSON con ambas secciones
json_data = {
    "sections": [contenido_sporting, contenido_manchester]
}

# Convierte el diccionario a JSON
json_output = json.dumps(json_data, ensure_ascii=False, indent=4)

# Imprime o guarda el JSON estructurado
print(json_output)

## Encabezados

In [ ]:
import requests
from bs4 import BeautifulSoup
import json

# Paso 1: Obtén el número de sección de "Carrera en clubes"
url = "https://es.wikipedia.org/w/api.php"
params = {
    "action": "parse",
    "page": "Cristiano Ronaldo",
    "format": "json",
    "prop": "sections"  # Solicitamos solo las secciones
}

response = requests.get(url, params=params)
if response.status_code == 200:
    data = response.json()
    
    # Encuentra la sección de "Carrera en clubes"
    section_number = None
    for section in data["parse"]["sections"]:
        if section["line"] == "Carrera en clubes":
            section_number = section["index"]
            break

    # Si encontramos el número de sección, hacemos la solicitud para obtener su contenido
    if section_number:
        params = {
            "action": "parse",
            "page": "Cristiano Ronaldo",
            "format": "json",
            "prop": "text",
            "section": section_number
        }

        response = requests.get(url, params=params)
        
        if response.status_code == 200:
            data = response.json()
            content_html = data["parse"]["text"]["*"]
            
            # Paso 2: Analiza el HTML y estructura los datos
            soup = BeautifulSoup(content_html, "html.parser")
            structured_data = {"Carrera en clubes": {}}
            
            # Encuentra todos los subtítulos (h4) en la sección "Carrera en clubes"
            for header in soup.find_all("h4"):
                section_title = header.get_text(strip=True)
                paragraphs = []
                
                # Extrae los párrafos asociados a esta subsección
                for sibling in header.find_next_siblings():
                    if sibling.name == "h4":
                        break  # Detente en la siguiente subsección
                    if sibling.name == "p":
                        paragraphs.append(sibling.get_text(strip=True))

                # Agrega la subsección y su contenido al diccionario
                structured_data["Carrera en clubes"][section_title] = " ".join(paragraphs)

            # Convierte el diccionario en JSON y lo muestra
            structured_data_json = json.dumps(structured_data, indent=4, ensure_ascii=False)
            print(structured_data_json)
        else:
            print("Error al obtener el contenido de la sección.")
    else:
        print("No se encontró la sección 'Carrera en clubes'.")
else:
    print(f"Error al obtener las secciones: {response.status_code}")

## section 3

In [ ]:
import requests

# Configura los parámetros para la solicitud
url = "https://es.wikipedia.org/w/api.php"
params = {
    "action": "parse",              # Utiliza la acción "parse" para obtener contenido de secciones
    "page": "Cristiano Ronaldo",    # Página de la cual obtener información
    "format": "json",               # Formato de respuesta
    "prop": "text",                 # Propiedad para obtener el contenido en HTML
    "section": 3                    # Número de la sección (3 corresponde a "Carrera en clubes" en esta página)
}

# Realiza la solicitud
response = requests.get(url, params=params)

# Verifica el código de respuesta
if response.status_code == 200:
    data = response.json()
    content_html = data["parse"]["text"]["*"]  # Extrae el HTML de la sección
    print(content_html)
else:
    print(f"Error en la solicitud: {response.status_code}")